In [ ]:
!pip3 install chart-studio 

     |████████████████████████████████| 71kB 5.0MB/s 


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving data_job.txt to data_job.txt


In [ ]:
%matplotlib inline

import numpy as np
import math
import time
import random
import pickle
import plotly
import chart_studio
import chart_studio.plotly as py
import plotly.graph_objs as go

# Enter your own plotly credentials to play on plots
chart_studio.tools.set_credentials_file(username='', api_key='')

data = []
f = open("data_job.txt", 'r')
temp = []
for a in f:
  d = a.split()
  if (len(d) != 0):
    temp.append(d)
  else:
    data.append(temp)
    temp = []
data.append(temp)
temp = []
print(data)
#Матрица возможности выйти на смену
matrixProb = []
for i in data[0]:
  t = []
  for k in i:
    t.append(int(k))
  matrixProb.append(t)
#Матрица требуемых людей на смену
matrixCount = []
for i in data[1][0]:
  matrixCount.append(int(i))
#Матрица степени квалифицированности сотрудника
matrixQual = []
for i in data[2][0]:
  matrixQual.append(int(i))
#Матрица максимальных затрат работников
matrixMaxJob = []
for i in data[3][0]:
  matrixMaxJob.append(int(i))
#Матрица оплаты работы
matrixPayJob = []
for i in data[4][0]:
  matrixPayJob.append(int(i))
print('Матрица возможности выйти на смену', matrixProb)
print('Матрица требуемых людей на смену', matrixCount)
print('Матрица степени квалифицированности сотрудника', matrixQual)
print('Матрица максимальных затрат сотрудника', matrixMaxJob)
print("Матрица цены смены работника", matrixPayJob)
startResult = []
for i in range(0,len(matrixCount)):
  startResult.append([])
  for d in range(0,matrixCount[i]):
    startResult[i].append(-1)
print("Матрица стартовых значений позиций смен", startResult)


#Адаптация алгоритма получения решения под отжиг
def canBeFitted(daycount, manNumber,matrixFit,matrixQualityFit):
  if matrixFit[daycount] != 0:
    if (matrixProb[manNumber][daycount] != 0):
      if matrixQualityFit[daycount] == 0:
        if matrixQual[manNumber] == 1:
          return True
        else:
          return False
      else:
        return True
  return False
def checkwillOverwork(manNumber, value,matrixJobFit):
  if (matrixJobFit[manNumber] - value < 0):
    return True
  else:
    return False
def wasAlreadyFitted(daycount,manNumber,result):
  #print(len(result[daycount]))
  a = 0
  while a < len(result[daycount]):
    #print(a)
    if manNumber == result[daycount][a]:
      return True
    a += 1
  return False
def wasPrevioslyFitted(daycount, manNumber, matrixSolutions):
  for a in matrixSolutions:
    for i in a[daycount]:
      if i == manNumber:
        #print(i, i == manNumber)
        return True
  return False


def generateSolution(prevSolutions,matrixCount,matrixMaxJob,matrixQual,ad):
  #Количество смен
  count = 3
  #Длительность 1 смены
  timerVal = 8
  #Матрица-определитель количества оставшегося количества требуемых на смену людей
  matrixFit = matrixCount[:]
  #Матрица-определитель количества оставшихся у каждого работника часов выработки до переработки
  matrixJobFit = matrixMaxJob[:]
  #print(matrixFit)
  matrixQualityFit = [0] * count
  i = 0
  result = ad[:]
  while i < count:
    for d in range(0, len(result[i])):
      for a in range(0,3):
        #print(matrixProb[a][i], matrixQual[a])
        cbf = canBeFitted(i, a,matrixFit,matrixQualityFit)
        waf = wasAlreadyFitted(i,a,result)
        wow = checkwillOverwork(a, 8,matrixJobFit)
        if (not  wasPrevioslyFitted(i, a ,prevSolutions)):
          if (cbf and not waf and not wow):
            #print(i,a)
            result[i][d] = a
            matrixQualityFit[i] += matrixQual[a]
            matrixFit[i] -= 1
            matrixJobFit[a] -= 8
            break
    i += 1
  return [result, matrixFit, matrixJobFit]


def getDeCost(solutionFactor, cost):
  result = 0
  for a in solutionFactor:
    for b in a:
      if (b == -1):
        result += 1
  return result * -cost
def getAddition(solutionFactor, cost):
  result = 0
  for a in solutionFactor:
    for b in a:
      if (b != -1):
        result += 1
  return result * cost
def getPotential(solutionFactor, cost):
  result = 0
  for a in solutionFactor:
    for b in a:
        result += 1
  return result * cost
def printTest(s):
  print("Потенциальный доход ",getPotential(s, 100), " Доход ", getAddition(s, 100), " Убыток ", getDeCost(s, 100))

#Функция определения решения для перехода
def neighbour(matrixSolutions):
  startResult = []
  for i in range(0,len(matrixCount)):
    startResult.append([])
    for d in range(0,matrixCount[i]):
      startResult[i].append(-1)
  return generateSolution(matrixSolutions, matrixCount, matrixMaxJob,matrixQual,startResult)[0]

#Алгоритм отжига
def E(s, c):
  return -getDeCost(s, c)
def P(e_new,e_old,T):
  return math.exp(-1 * (e_new - e_old) / T)
def temperature(arg,Targ):
  return arg/Targ
def anneal(k_max, matrixSolutions):
  Timing = []
  Tseq = []
  Sseq = []
  s0 = neighbour(matrixSolutions)
  
  T = random.randint(0,10)
  s = s0
  k = 1
  while k <= k_max:
      Timing.append(k)
      Sseq.append(s)
      Tseq.append(T)

      T = temperature(k / k_max,T)
      s_new = neighbour(matrixSolutions)
    #print(s_new)
      e_old = E(s,5)
      e_new = E(s_new,5)
      if P(e_old, e_new, T) >= random.randint(0, 1):
          s = s_new
      k += 1
      #print(s)
  return [s,Sseq,Tseq,Timing]
#Доп функции
#Подсчет цены решения
#Эффективность считаем в убытке. Если не нашли человека на позицию в смене это повышение цены(убыток)
#def isBetter(solution, solutionList):
matrixSolutions = []
import plotly.express as px
Tseq = []
testSeq = []
timing = []
l = anneal(100, matrixSolutions)
testSeq = l[1]
Tseq = l[2]
timing = l[3]
result = l[0]
print(result)

[[['0', '1', '1'], ['0', '0', '1'], ['1', '0', '1']], [['2', '1', '2']], [['1', '1', '1']], [['20', '20', '20']], [['10', '15', '20']]]
Матрица возможности выйти на смену [[0, 1, 1], [0, 0, 1], [1, 0, 1]]
Матрица требуемых людей на смену [2, 1, 2]
Матрица степени квалифицированности сотрудника [1, 1, 1]
Матрица максимальных затрат сотрудника [20, 20, 20]
Матрица цены смены работника [10, 15, 20]
Матрица стартовых значений позиций смен [[-1, -1], [-1], [-1, -1]]
[[2, -1], [0], [0, 1]]


In [ ]:
df = Tseq
fig = px.line(df, x=timing, y=Tseq, title='Test')
fig.show()